# ROC and PR Curves

### Preliminary Plots
Before diving into the receiver operating characteristic (ROC) curve, we will look at two plots that will give some context to the thresholds mechanism behind the ROC and PR curves.

In the histogram, we observe that the score spread such that most of the positive labels are binned near 1, and a lot of the negative labels are close to 0. When we set a threshold on the score, all of the bins to its left will be classified as 0's, and everything to the right will be 1's. There are obviously a few outliers, such as **negative** samples that our model gave a high score, and positive samples with a low score. If we set a threshold right in the middle, those outliers will respectively become **false positives** and false negatives.

As we adjust thresholds, the number of positive positives will increase or decrease, and at the same time the number of true positives will also change; this is shown in the second plot. As you can see, the model seems to perform fairly well, because the true positive rate decreases slowly, whereas the false positive rate decreases sharply as we increase the threshold. Those two lines each represent a dimension of the ROC curve.

In [1]:
using DataFrames
using PlotlyJS
using ScikitLearn
@sk_import datasets: make_classification
@sk_import linear_model: LogisticRegression
@sk_import metrics: (roc_curve, auc)

X, y = make_classification(n_samples=500, random_state=0)

model = LogisticRegression()
model.fit(X,y)
y_score = model.predict_proba(X)[:, 1]
fpr, tpr, thresholds = roc_curve(y, y_score)

df_hist = DataFrame([y_score, y], ["y_score", "y_true"])


# The histogram of scores compared to true labels
plot(df_hist, x=:y_score, color=:y_true, kind="histogram", nbinsx=50,Layout(barmode="stack"))

WebIO._IJuliaInit()

data: [
  "histogram with fields legendgroup, marker, name, nbinsx, showlegend, type, x, xaxis, and yaxis",
  "histogram with fields legendgroup, marker, name, nbinsx, showlegend, type, x, xaxis, and yaxis"
]

layout: "layout with fields barmode, legend, margin, template, and xaxis"

In [2]:
# Evaluating model performance at various thresholds
df = DataFrame(["Thresholds" => thresholds, "False_Positive_Rate" => fpr, :"True_Positive_Rate" => tpr])

trace0 = scatter(
    df,
    x=:Thresholds,
    y=:False_Positive_Rate,
    name="False Positive Rate"
)

trace1 = scatter(
    df,
    x=:Thresholds,
    y=:True_Positive_Rate,
    name="True Positive Rate"
)

plot(
    [trace0, trace1],
    Layout(
        title="TPR and FPR at every threshold",
        width=700,
        height=500,
        xaxis=attr(
            title="Thresholds",
            range=[0, 1],
            constrain="domain"
        ),
        yaxis=attr(
            scaleanchor="x",
            scaleratio=1
        )       
    )
)

data: [
  "scatter with fields name, type, x, and y",
  "scatter with fields name, type, x, and y"
]

layout: "layout with fields height, margin, template, title, width, xaxis, and yaxis"

### Basic binary ROC curve
Notice how this ROC curve looks similar to the True Positive Rate curve from the previous plot. This is because they are the same curve, except the x-axis consists of increasing values of FPR instead of threshold, which is why the line is flipped and distorted.

We also display the area under the ROC curve (ROC AUC), which is fairly high, thus consistent with our interpretation of the previous plots.

In [1]:
using DataFrames
using PlotlyJS
using ScikitLearn
@sk_import datasets: make_classification
@sk_import linear_model: LogisticRegression
@sk_import metrics: (roc_curve, auc)

X, y = make_classification(n_samples=500, random_state=0)

model = LogisticRegression()
model.fit(X, y)
y_score = model.predict_proba(X)[:, 2]

fpr, tpr, thresholds = roc_curve(y, y_score)

fig = plot(
    scatter(
        x=fpr,
        y=tpr,
        fill="tozeroy",
        width=700, height=500
    ),
    Layout(
        title_text="ROC Curve (AUC=$((auc(fpr, tpr))))",
        xaxis=attr(constrain="domain"),
        yaxis=attr(
            scaleanchor="x",
            scaleratio=1
        )
    )
)

add_shape!(
    fig,
    line(
        x0=0, x1=1, y0=0, y1=1,
        line=attr(dash="dash")
    )
)

fig

WebIO._IJuliaInit()

data: [
  "scatter with fields fill, height, type, width, x, and y"
]

layout: "layout with fields margin, shapes, template, title, xaxis, and yaxis"

### ROC curve in Dash

In [ ]:
using PlotlyJS
using Dash, DashCoreComponents, DashHtmlComponents
using ScikitLearn
using ScikitLearn.CrossValidation: train_test_split
@sk_import linear_model: LogisticRegression
@sk_import datasets: (load_iris, make_classification)
@sk_import metrics: (roc_curve, auc)
@sk_import tree: DecisionTreeClassifier
@sk_import neighbors: KNeighborsClassifier

X, y = make_classification(n_samples=1500, random_state=0)
X_train, X_test, y_train, y_test = train_test_split(
    X, y, random_state=42)

MODELS=Dict("Logistic" .=> LogisticRegression(),
            "Decision Tree" .=> DecisionTreeClassifier(),
            "k-NN" .=> KNeighborsClassifier())

app = dash()

app.layout = html_div(
    [
        html_p("Train Model:"),
        dcc_dropdown(
            id="model-name",
            options=[(label=k, value=k) for (k,v) in MODELS],
            value="Logistic",
            clearable=false
        ),
        dcc_graph(id="graph")
    ]
)

callback!(
    app,
    Output("graph", "figure"),
    Input("model-name", "value")
) do name 
    model = MODELS[name]
    model.fit(X_train, y_train)
    
    y_score = model.predict_proba(X)[:, 2]
    fpr, tpr, thresholds = roc_curve(y, y_score)
    score = auc(fpr, tpr)

    fig = plot(
        scatter(
            x=fpr,
            y=tpr,
            fill="tozeroy",
        ),
        Layout(
            title_text="ROC Curve (AUC=$(score))",
            xaxis_title="False Positive Rate",
            yaxis_title="True Positive Rate"           
        )
    )

    add_shape!(
    fig,
        line(
            x0=0, x1=1, y0=0, y1=1,
            line=attr(dash="dash")
        )
    )

    return fig
end

run_server(app, "0.0.0.0", debug=false)

### Multiclass ROC Curve
<span style="color:blue">TODO: add noise to make task harder</span>

In [1]:
using DataFrames
using PlotlyJS
using ScikitLearn
@sk_import datasets: (load_iris, make_classification)
@sk_import linear_model: LogisticRegression
@sk_import metrics: (roc_curve, auc, roc_auc_score)

df = load_iris()
X = df["data"][:, :]  # we only take the first two features.
y = df["target"]
y=y[:,:]
model = LogisticRegression(max_iter=200)
model.fit(X, y)
y_scores = model.predict_proba(X);
y=DataFrame(y, ["target"]);

# One hot encode the labels in order to plot them
y_onehot = select(y, [:target => ByRow(isequal(v)).=> Symbol(v) for v in unique(y.target)]);
rename!(y_onehot,[:1, :2, :3] .=>  [:setosa, :versicolor, :virginica] );

# Create an empty trace list, and iteratively add new lines
# every time we compute a new class
traces=[]

for i in 1:size(y_scores)[2]
    y_true = y_onehot[:, i]
    y_score = y_scores[:, i]

    fpr, tpr, _ = roc_curve(y_true, y_score)
    auc_score = roc_auc_score(y_true, y_score)
    
    push!(
        traces,
        scatter(
            x=fpr,
            y=tpr,
            name=names(y_onehot)[i]
        )
    ) 
end

fig = plot(
    [trace for trace in traces],
    Layout(
        xaxis_title="False Positive Rate",
        yaxis_title="True Positive Rate",
        yaxis=attr(scaleanchor="x", scaleratio=1),
        xaxis=attr(constrain="domain"),
        width=700, height=500
    )
)

add_shape!(
    fig,
    line(
        x0=0, x1=1, y0=0, y1=1,
        line=attr(dash="dash")
    )
)

fig

WebIO._IJuliaInit()

C:\Users\CEPEL\.julia\conda\3\lib\site-packages\sklearn\utils\validation.py:63: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  return f(*args, **kwargs)


data: [
  "scatter with fields name, type, x, and y",
  "scatter with fields name, type, x, and y",
  "scatter with fields name, type, x, and y"
]

layout: "layout with fields height, margin, shapes, template, width, xaxis, and yaxis"

## Precision-Recall Curves
<span style="color:blue">TODO: Dynamic title</span>

In [1]:
using DataFrames
using PlotlyJS
using ScikitLearn
@sk_import datasets: (load_iris, make_classification)
@sk_import linear_model: LogisticRegression
@sk_import metrics: (roc_curve, auc, precision_recall_curve)

X, y = make_classification(n_samples=500, random_state=0)

model = LogisticRegression()
model.fit(X,y)
y_score = model.predict_proba(X)[:, 2]

precision, recall, thresholds = precision_recall_curve(y, y_score)

fig = plot(
    scatter(
        x=recall,
        y=precision,
        fill="tozeroy",
        width=700, height=500
    ),
    Layout(
        title="Precision-Recall Curve",
        xaxis=attr(title="Recall", constrain="domain"),
        yaxis=attr(
            title="Precision",
            scaleanchor="x",
            scaleratio=1
        )
    )
)

add_shape!(
    fig,
    line(
        x0=0, x1=1, y0=0, y1=1,
        line=attr(dash="dash")
    )
)

fig

WebIO._IJuliaInit()

data: [
  "scatter with fields fill, height, type, width, x, and y"
]

layout: "layout with fields margin, shapes, template, title, xaxis, and yaxis"

### Average Precision

In [4]:
using DataFrames
using PlotlyJS
using ScikitLearn
@sk_import datasets: (load_iris, make_classification)
@sk_import linear_model: LogisticRegression
@sk_import metrics: (precision_recall_curve, average_precision_score)

df = load_iris()
X = df["data"][:, :]  # we only take the first two features.
y = df["target"]
y=y[:,:]
model = LogisticRegression(max_iter=200)
model.fit(X, y)
y_scores = model.predict_proba(X);
y=DataFrame(y, ["target"]);

# One hot encode the labels in order to plot them
y_onehot = select(y, [:target => ByRow(isequal(v)).=> Symbol(v) for v in unique(y.target)]);
rename!(y_onehot,[:1, :2, :3] .=>  [:setosa, :versicolor, :virginica] );

# Create an empty trace list, and iteratively add new lines
# every time we compute a new class
traces=[]

for i in 1:size(y_scores)[2]
    y_true = y_onehot[:, i]
    y_score = y_scores[:, i]

    precision, recall, _ = precision_recall_curve(y_true, y_score)
    auc_score = average_precision_score(y_true, y_score)
    
    push!(
        traces,
        scatter(
            x=recall,
            y=precision,
            name=names(y_onehot)[i]
        )
    ) 
end

fig = plot(
    [trace for trace in traces],
    Layout(
        xaxis_title="Recall",
        yaxis_title="Precision",
        yaxis=attr(scaleanchor="x", scaleratio=1),
        xaxis=attr(constrain="domain"),
        width=700, height=500
    )
)

add_shape!(
    fig,
    line(
        x0=0, x1=1, y0=1, y1=0,
        line=attr(dash="dash")
    )
)

fig

C:\Users\CEPEL\.julia\conda\3\lib\site-packages\sklearn\utils\validation.py:63: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  return f(*args, **kwargs)


data: [
  "scatter with fields name, type, x, and y",
  "scatter with fields name, type, x, and y",
  "scatter with fields name, type, x, and y"
]

layout: "layout with fields height, margin, shapes, template, width, xaxis, and yaxis"